In [5]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
sys.path.append("..") #把上级目录加入到变量中
from langchain import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
import os
import logging
from config import OPENAI_API_KEY, FAISS_DB_PATH
from query_data import get_chain, get_citations


sk = "sk"
dash = "-"
my_key= "VgaqoS0BbjxbVN7RFWcJT3BlbkFJYy3W265nnshPERsxhFMy"
os.environ['OPENAI_API_KEY'] = sk + dash + my_key

In [54]:
# Load from existing index
rds = FAISS.load_local("../db", OpenAIEmbeddings())

In [74]:
query = "星空如何设置单机版纸电一体开票服务？"
results = rds.similarity_search_with_score(query)
for r in results:
    print(r[0].page_content + "\n ========================")

星空支持全电发票须知

目前星空企业版有两种途径支持全电发票：1、https://www.yuque.com/piaozone/rou5ks/ei5cmu4z9vsm241g?singleDoc# 《星空升级全电配置（RPA）》

星空无需改动，在发票云公众号申请升级后简单配置即可，升级后在星空中点击【电子发票开具】按钮会自动开具全电发票

不支持特殊行业全电发票开票，不支持同一时间并行开具税控发票和全电发票，需手工切换设置

适用版本：

金蝶云版本（星空）：7.6及以上版本

配套所购金蝶云模块要求：应收模块+发票管理模块

2、https://www.yuque.com/piaozone/rou5ks/qlz93cbz0qylr1he?singleDoc# 《星空企业版补丁包——全电开票实施文档（RPA）》

系统集成星瀚发票云，需要打补丁包，将原有星空开票单据替换为星瀚发票云单据，需要走商务以及实施

支持特殊行业全电发票开票，支持同一时间并行开具税控发票和全电发票

适用版本：

星空7.2及以上私有云版本、公有云单租户版本（销售增值税发票以及金税开票单无二开过）

配套所购金蝶云模块要求：应收模块+发票管理模块

请按照实际情况选择对接方式，谢谢。
金蝶云星空【单机版纸电一体开票服务】实施文档

1.   实施说明

1.1发票云产品对接ERP产品线要求清单

1、金蝶云版本：7.3以上版本  2、配套所购金蝶云模块要求：应收模块+发票管理模块3、金蝶云补丁：建议打上金蝶云最新的补丁（建议安装Cloud最新的补丁，金税对接的功能是持续完善的，并且Cloud的税收分类编码目前是在补丁中升级，所以建议经常打最新补丁，以便获得较好的使用体验）

1.2实施流程

实施前请先确认清楚税盘类型（航信/百望），ERP版本（例如金蝶云7.3），发票云产品类型，约定产品使用的具体场景，以便于后续实施工作的开展。发票云纸电一体实施流程如下：

2.发票云产品激活

产品出库后，发票云将发送一封激活邮件至填写的出库邮箱地址，按照操作提示完成激活即可。 详细激活流程可参考（必看）：https://www.yuque.com/docs/share/dabc5f5a-beac-472f-ad2e-4d9eda96da85?# 《发票云产品激活说明》  
所有激活工作完成后，将会获

In [56]:
retriever = rds.as_retriever()
# retriever.search_kwargs['k'] = 2
chatbot = get_chain(retriever)
# create a chat history buffer
chat_history = []

In [75]:
# gather user input for the first question to kick off the bot
query = "星空如何设置单机版纸电一体开票服务？"

result = chatbot({"question": query, "chat_history": chat_history})

Sorry，发票云知识库找不到合适答案。我会补充来自ChatGPT的回应...

In [78]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever, return_source_documents=True)

In [82]:
# ret = qa({"query": "黑客帝国有几部？"})
ret = qa({"query": query})
f"\n更多详情，请参考：{get_citations(ret['source_documents'])}\n"

'\n更多详情，请参考：[星空企业版补丁包——全电开票实施文档（乐企）：https://www.yuque.com/piaozone/implement/qvox7ovri35qd1s2] [金蝶云星空【单机版纸电一体开票服务】实施文档：https://www.yuque.com/piaozone/implement/ncb74y] [星空企业版补丁包——全电平台收票实施文档：https://www.yuque.com/piaozone/implement/gzgdq2ukskigi714] [星空支持全电发票须知：https://www.yuque.com/piaozone/implement/aiwgkrqlgcet11xh]\n'